In [ ]:
# imports
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline

In [ ]:
# Settings for displaying floats
pd.set_option('display.float_format', '{:,.2f}'.format)

In [ ]:
df = pd.read_csv("/Users/mareikekeller/air_quality/data/cleaned_data.csv")
df.head()

In [ ]:
df["City"].nunique()

In [ ]:
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Split 'Date' column into 'year', 'month' and 'day'
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

# Remove 'Date' column
if 'Date' in df.columns:
    df.drop(columns=['Date'], inplace=True)

In [ ]:
# Daten für 2014 & 2025 entfernen, weil zu wenige Datenpunkte
df = df[(df["year"] > 2014) & (df["year"] < 2025)]

In [ ]:
# Display the first 5 rows of the dataframe
df.head()

In [ ]:
df.shape

In [ ]:
# Berechnen, wie viele Prozent der Werte pro Spalte fehlen
missing_percent = df.isna().mean() * 100  

# Spalten auswählen, die weniger als 50% fehlende Werte haben
df_cleaned = df.loc[:, missing_percent <= 53]

# Ergebnis ausgeben
print(f"Anzahl der entfernten Spalten: {df.shape[1] - df_cleaned.shape[1]}")
print("Übrige Spalten:", df_cleaned.columns)


In [ ]:
df_cleaned["City"].nunique()

In [ ]:
# Liste der Schadstoff-Features für das Clustering
pollutants = ["co", "no2", "o3", "so2", "pm10", "pm25"]

In [ ]:
# Mit Heatmap herausfinden, welche Städte für welche Schadstoffe fehlende Werte haben

# DataFrame mit den Schadstoffen pro Stadt erstellen
df_missing = df_cleaned.groupby("City")[pollutants].mean()

# Boolean-Maske für fehlende Werte erstellen (True = fehlend, False = vorhanden)
missing_data = df_missing.isna()

# Größe der Grafik anpassen
plt.figure(figsize=(8, 20))

# Heatmap zeichnen (dunklere Farben = mehr fehlende Werte)
sns.heatmap(missing_data, cmap="coolwarm", cbar=False, linewidths=0.5)

# Achsentitel setzen
plt.xlabel("Schadstoffe")
plt.ylabel("Städte")
plt.title("Heatmap der fehlenden Werte pro Stadt und Schadstoff")


In [ ]:
missing_per_city = df_missing.isna().sum(axis=1)
missing_per_city_sorted = missing_per_city.sort_values(ascending=False)
print(missing_per_city_sorted.to_string())


In [ ]:
# Clusteranalyse zur Schadstoffbelastung

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Liste der Schadstoff-Features für das Clustering
pollutants = ["co", "no2", "o3", "so2", "pm10", "pm25"]

# Durchschnittliche Schadstoffwerte pro Stadt berechnen
df_cluster = df_cleaned.groupby("City")[pollutants].mean().dropna()



In [ ]:
df_cluster.head()

In [ ]:
df_cluster.shape

In [ ]:
# Daten skalieren (K-Means ist empfindlich gegenüber unterschiedlichen Skalen)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

# Ergebnis als DataFrame zurückgeben
df_cluster_scaled = pd.DataFrame(df_scaled, index=df_cluster.index, columns=pollutants)

# Überprüfen, ob die Daten korrekt vorbereitet sind
df_cluster_scaled.head()

In [ ]:
# Teste verschiedene Clusterzahlen (k = 1 bis 10)
inertia = []
k_values = range(1, 31)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_cluster_scaled)
    inertia.append(kmeans.inertia_)  # Speichert den Fehler (Inertia)

# Elbow-Plot erstellen
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia, marker='o', linestyle='-')
plt.xlabel("Anzahl der Cluster (k)")
plt.ylabel("Inertia (Fehler)")
plt.title("Elbow-Methode zur Bestimmung der optimalen Clusterzahl")
plt.grid(True);

In [ ]:
# K-Means-Clustering
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
df_cluster_scaled["Cluster"] = kmeans.fit_predict(df_cluster_scaled)

# Neue Cluster-Zuordnung der Städte anzeigen
df_clusters_no_outliers = df_cluster_scaled[["Cluster"]].sort_values(by="Cluster")
df_clusters_no_outliers

In [ ]:
df_clusters_no_outliers.shape

Entscheidung: 5 Cluster sind gut - aber Teheran it so dreckig, dass es allein sein eigens Cluster bildet. Also nehmen wir es erst mal raus, damit es die anderen Cluster nicht verzerrt. Dadurch nehmen wir für die Clusteranalyse aber nur noch 4 Cluster.

In [ ]:
# Teheran aus dem DataFrame entfernen
df_cluster_no_tehran = df_cluster_scaled.drop(index="Tehran")

# K-Means erneut ausführen mit 4 Clustern (oder einer anderen optimalen Zahl)
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_cluster_no_tehran["Cluster"] = kmeans.fit_predict(df_cluster_no_tehran)

# Neue Cluster-Zugehörigkeit der Städte anzeigen
df_clusters_no_tehran = df_cluster_no_tehran[["Cluster"]]
df_clusters_no_tehran_sorted = df_clusters_no_tehran.sort_values(by="Cluster")  # Sortiert nach Cluster
df_clusters_no_tehran_sorted

In [ ]:
df_clusters_no_tehran_sorted.shape

In [ ]:
type(df_clusters_no_tehran_sorted)

In [ ]:
# Cluster-Labels zu df_cluster hinzufügen
df_clusters_no_tehran_sorted["Cluster"] = kmeans.labels_

# Merge mit ursprünglichem DataFrame
df = df.merge(df_clusters_no_tehran_sorted[["Cluster"]], left_on="City", right_index=True, how="left")

# Überprüfen, ob die Cluster-Zuordnung funktioniert hat
df.head(20)
